In [1]:
from transformers import BartTokenizer, BartForQuestionAnswering
import torch



/opt/conda/envs/main/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
# Initialize the BART tokenizer and model
tokenizer = BartTokenizer.from_pretrained( "valhalla/bart-large-finetuned-squadv1")
model = BartForQuestionAnswering.from_pretrained( "valhalla/bart-large-finetuned-squadv1")
# Define the context, question, and answer
context = "The capital of France is Paris."
question = "What is the capital of France?"
answer = "Paris"

context_tokens = tokenizer(context, return_tensors='pt')['input_ids'].view(-1)
question_tokens = tokenizer(question, return_tensors='pt')['input_ids'].view(-1)
answer_tokens = tokenizer(answer, return_tensors='pt')['input_ids'].view(-1)

print(f"Context tokens: {context_tokens}, \n"
      f"Question tokens: {question_tokens}, \n"
      f"Answer tokens: {answer_tokens}, \n",
      f"Padding token id: {tokenizer.pad_token_id}, \n",
      f"BOS token id: {tokenizer.bos_token_id}, \n",
      f"EOS token id: {tokenizer.eos_token_id}, \n",)

combined_tokens = torch.cat([context_tokens[:-1], question_tokens[1:-1], answer_tokens[1:]], dim=0)
context_question_tokens = torch.cat([context_tokens[:-1], question_tokens[1:]], dim=0).unsqueeze(0)
answer_start_idx = len(context_tokens) - 1 + len(question_tokens) - 2
answer_end_idx = answer_start_idx + len(answer_tokens) - 2

combined_tokens = combined_tokens.unsqueeze(0)
answer_start_idx = torch.tensor([answer_start_idx]).unsqueeze(0)
answer_end_idx = torch.tensor([answer_end_idx]).unsqueeze(0)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


Context tokens: tensor([   0,  133,  812,    9, 1470,   16, 2201,    4,    2]), 
Question tokens: tensor([   0, 2264,   16,    5,  812,    9, 1470,  116,    2]), 
Answer tokens: tensor([    0, 32826,     2]), 
 Padding token id: 1, 
 BOS token id: 0, 
 EOS token id: 2, 



In [25]:
# Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

# Perform a forward pass and compute the loss
model.train()
outputs = model(input_ids=combined_tokens, start_positions=answer_start_idx, end_positions=answer_end_idx)
loss = outputs.loss

# Backpropagate the loss and perform an optimization step
loss.backward()
optimizer.step()

# Perform inference on the same data
model.eval()
with torch.no_grad():
    outputs = model(input_ids=context_question_tokens)

for key, value in outputs.__dict__.items():
    output_string = f"{key}: {value}" if not isinstance(value, torch.Tensor) else f"{key}: {value.shape}"
    # print(output_string)

encoder_outputs
{
    'last_hidden_state': tensor([[[ 0.0150,  0.0184, -0.0024,  ...,  0.0036, -0.0229, -0.0058],
         [-0.2851,  0.0260, -0.3642,  ..., -0.2097, -0.0389, -0.0300],
         [-0.1509,  0.0110,  0.1628,  ..., -0.1412,  0.0178, -0.0624],
         ...,
         [-0.1158, -0.1886,  0.1772,  ..., -0.0938,  0.0355, -0.0410],
         [-0.1504, -0.2321,  0.0447,  ...,  0.1199,  0.1284,  0.0939],
         [ 0.0927,  0.1256, -0.0512,  ..., -0.0361, -0.1075,  0.1066]]],
       grad_fn=<NativeLayerNormBackward0>),
    'hidden_states': None,
    'attentions': None
}

last_hidden_state: torch.Size([1, 17, 1024])

hidden_states: None

attentions: None

encoder_outputs
{
    'last_hidden_state': tensor([[[ 0.0113,  0.0185,  0.0048,  ..., -0.0053, -0.0167, -0.0071],
         [-0.2570, -0.0737, -0.3358,  ..., -0.1560,  0.0068, -0.0128],
         [-0.1610,  0.0081,  0.1093,  ..., -0.0902,  0.0416, -0.0264],
         ...,
         [-0.1389,  0.0329, -0.1662,  ..., -0.1552, -0.0731,  0.2212],
         [-0.1460, -0.2380,  0.1070,  ..., -0.0674, -0.0898, -0.1011],
         [ 0.0747,  0.0827, -0.0550,  ..., -0.0324, -0.1199,  0.1103]]]),
    'hidden_states': None,
    'attentions': None
}

last_hidden_state: torch.Size([1, 16, 1024])

hidden_states: None

attentions: None

In [44]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW, get_linear_schedule_with_warmup
import torch
from tqdm.auto import tqdm

# Initialize the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Set up the question-answer pair as a single string
question = "What is the capital of France?"
answer = "The capital of France is Paris."
text = question + " " + answer

# Encode the text, and return tensors
inputs = tokenizer.encode_plus(text, return_tensors="pt", add_special_tokens=True)

# Extract the input_ids and attention_mask
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

# Set up the optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=1e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=1)

# Move the model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.train()
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
for i in tqdm(range(100)):
    # Zero the gradients
    optimizer.zero_grad()
    # Forward pass
    outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)

    # Get the loss
    loss = outputs.loss

    # Backward pass
    loss.backward()

    # Update weights
    optimizer.step()

    # Update the learning rate
    scheduler.step()
    print(loss.item())

# Do a prediction (inference)
model.eval()

# Let's ask the model the same question
input_ids = tokenizer.encode(question, return_tensors="pt").to(device)

# Generate a response
generated = model.generate(input_ids, max_length=50, num_beams=5, temperature=1.5, no_repeat_ngram_size=2)
generated_answer = tokenizer.decode(generated[:, input_ids.shape[-1]:][0], skip_special_tokens=True)

print(f"Generated answer: {generated_answer}")


/opt/conda/envs/main/lib/python3.10/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  3%|▎         | 3/100 [00:00<00:04, 20.35it/s]

3.5292856693267822
2.5621626377105713
2.5013720989227295
2.8336524963378906
2.499972343444824


  8%|▊         | 8/100 [00:00<00:04, 19.78it/s]

2.569439172744751
2.568347454071045
2.5246505737304688
2.4872758388519287
2.0676064491271973


 14%|█▍        | 14/100 [00:00<00:04, 19.59it/s]

...

2.461033582687378
2.6076159477233887
2.9964075088500977
2.7833333015441895


 97%|█████████▋| 97/100 [00:05<00:00, 19.39it/s]

2.504783868789673
2.8673207759857178
2.2754602432250977
3.0448973178863525


100%|██████████| 100/100 [00:05<00:00, 18.02it/s]
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


2.2931196689605713
2.8419203758239746
Generated answer: 

The capital is Paris. It is located in the heart of the French capital. The capital was founded in 1789, and it is still there today. In 1791, the city was divided into two
